In [20]:
import geopandas as gpd
import os
import glob
import rasterio
from shapely.geometry import box

In [21]:
buildings = gpd.read_file("Raw_dataset/data_02.shp")

# Folder containing your TIFF tiles
tif_folder = "Dataset/train"
search_pattern = os.path.join(tif_folder, "Dataset.*.tif")
tile_paths = sorted(glob.glob(search_pattern))

# Output folder for clipped shapefiles
output_folder = "Dataset/train_shp"
os.makedirs(output_folder, exist_ok=True)

In [22]:
for tile_path in tile_paths:
    with rasterio.open(tile_path) as src:
        bounds = src.bounds
        tile_crs = src.crs
        tile_name = os.path.splitext(os.path.basename(tile_path))[0]

        # Create a GeoDataFrame from the tile's bounding box
        tile_geom = box(bounds.left, bounds.bottom, bounds.right, bounds.top)
        tile_gdf = gpd.GeoDataFrame({'geometry': [tile_geom]}, crs=tile_crs)

        # Reproject shapefile if needed
        if buildings.crs != tile_crs:
            buildings_proj = buildings.to_crs(tile_crs)
        else:
            buildings_proj = buildings

        # Clip buildings to the tile's bounding box
        clipped = gpd.overlay(buildings_proj, tile_gdf, how='intersection')

        # Save to a new shapefile
        out_path = os.path.join(output_folder, f"{tile_name}.shp")
        clipped.to_file(out_path)

In [23]:
input_folder = "Dataset/train_shp"
output_folder = "Dataset/train_shp"

In [24]:
os.makedirs(input_folder, exist_ok=True)

In [25]:
shapefiles = glob.glob(os.path.join(input_folder, "*.shp"))

In [26]:
for shp_path in shapefiles:
    # Read the shapefile
    gdf = gpd.read_file(shp_path)

    # Apply buffer (adjust distance as needed; negative = shrink)
    buffered = gdf.buffer(-0.5)

    # Create new GeoDataFrame with buffered geometries
    buffered_gdf = gpd.GeoDataFrame(geometry=buffered, crs=gdf.crs)

    # Create output file name
    base_name = os.path.basename(shp_path)
    output_path = os.path.join(output_folder, base_name)

    # Save buffered shapefile
    buffered_gdf.to_file(output_path)
